In [1]:
# !pip install roboflow
# !pip install optuna

# Step 1: Load the dataset from Roboflow (not augmented)
from roboflow import Roboflow
rf = Roboflow(api_key="BZTREFbhM5T8N26Kzvy3")
project = rf.workspace("fire-rqbio").project("fire-and-smoke-yikzn")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
from ultralytics import YOLO

def objective(trial):
    # Hyperparameters to search
    try:
        params = {
            "epochs": trial.suggest_int("epochs", 5, 100),
            "batch": trial.suggest_categorical("batch", [8, 16, 32]),
            
            # Augmentation
            "flipud": trial.suggest_float("flipud", 0.0, 0.5),
            "fliplr": trial.suggest_float("fliplr", 0.0, 0.5),
            "mosaic": trial.suggest_float("mosaic", 0.5, 1.0),
            "mixup": trial.suggest_float("mixup", 0.0, 0.3),
            "degrees": trial.suggest_float("degrees", 0.0, 20.0),
            "scale": trial.suggest_float("scale", 0.3, 0.7),
            "shear": trial.suggest_float("shear", 0.0, 5.0),
            "hsv_h": trial.suggest_float("hsv_h", 0.0, 0.2),
            "hsv_s": trial.suggest_float("hsv_s", 0.0, 0.5),
            "hsv_v": trial.suggest_float("hsv_v", 0.0, 0.5),
            "translate": trial.suggest_float("translate", 0.05, 0.15),

            # Optimizer & LR
            "lr0": trial.suggest_float("lr0", 5e-3, 1e-2, log=True),
            "lrf": trial.suggest_float("lrf", 0.01, 0.5),
            "momentum": trial.suggest_float("momentum", 0.8, 0.98),
            "weight_decay": trial.suggest_float("weight_decay", 0.0001, 0.001),
            "warmup_epochs": trial.suggest_int("warmup_epochs", 0, 5),
            "warmup_momentum": trial.suggest_float("warmup_momentum", 0.5, 0.95),
            "warmup_bias_lr": trial.suggest_float("warmup_bias_lr", 0.05, 0.5),

            # Loss
            "box": trial.suggest_float("box", 0.7, 0.8),
            "cls": trial.suggest_float("cls", 0.4, 0.6),
            "dfl": trial.suggest_float("dfl", 1.0, 2.0),

            # Regularization
            "dropout": trial.suggest_float("dropout", 0.0, 0.5),
            "patience": trial.suggest_int("patience", 5, 20)
        }

        # Load model
        model = YOLO('yolov8n.pt')

        # Train model with the sampled hyperparameters
        results = model.train(
            data='./fire and smoke.v3i.yolov8/data.yaml',
            **params,
            verbose=False
        )

        # Evaluate the model and return mAP@0.5 as the objective metric
        metrics = model.val()
        return metrics.box.map50  # or map for overall mAP
    except Exception as e:
        print(f"Trial failed: {str(e)}")
        return float('nan')  # Mark failed trials



In [3]:
import optuna

# Create/Load study with SQLite storage
study = optuna.create_study(
    study_name="fire_detection_study",
    direction="maximize",
    storage="sqlite:///optuna.db",  # Saved to local SQLite file
    load_if_exists=True
)

[I 2025-04-24 06:09:23,086] Using an existing study with name 'fire_detection_study' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=15, gc_after_trial=True)

New https://pypi.org/project/ultralytics/8.3.114 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.107  Python-3.12.7 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./fire and smoke.v3i.yolov8/data.yaml, epochs=45, time=None, patience=6, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train44, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.37208882540209803, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\train\labels.cache... 2712 images, 5 backgrounds, 1 corrupt: 100%|██████████| 2713/2713 [00:00<?, ?it/s]

train: WARNING  C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\train\images\fire-and-truck-desert-an-aerial-view-of-a-very-small-fire-on-the-side-of-a-road-in-a-rural-desert-landscape-a-green-tractor-stands-by-as-the-fire-burns_bdglz60p8__M0000_jpg.rf.b186e9bec055032ed74041cc43e162d1.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'C:\\Users\\G15\\Desktop\\FireDetectionSystem-main\\fire and smoke.v3i.yolov8\\train\\images\\fire-and-truck-desert-an-aerial-view-of-a-very-small-fire-on-the-side-of-a-road-in-a-rural-desert-landscape-a-green-tractor-stands-by-as-the-fire-burns_bdglz60p8__M0000_jpg.rf.b186e9bec055032ed74041cc43e162d1.jpg'



val: Scanning C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\valid\labels.cache... 728 images, 1 backgrounds, 0 corrupt: 100%|██████████| 728/728 [00:00<?, ?it/s]


Plotting labels to runs\detect\train44\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.009404447490554748' and 'momentum=0.8210571884948419' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0006692004054048497), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train44
Starting training for 45 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        728       1524      0.632      0.535      0.595      0.276

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      58/68      2.09G     0.1802      1.495      1.369         37        640: 100%|██████████| 170/170 [00:29<00:00,  5.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:05<00:00,  4.58it/s]

                   all        728       1524      0.621      0.535      0.585       0.27


Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      59/68      2.09G     0.1729      1.383      1.368         38        640:   8%|▊         | 13/170 [00:02<00:27,  5.69it/s]

In [4]:
import optuna.visualization as vis

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_parallel_coordinate(study).show()

In [5]:
best_trial = study.best_trial
print(f"Best mAP@0.5: {best_trial.value}")
print("Best params:")
for k, v in best_trial.params.items():
    print(f"  {k}: {v}")

Best mAP@0.5: 0.588118801311563
Best params:
  epochs: 88
  batch: 8
  flipud: 0.10238091235701546
  fliplr: 0.23265006550845513
  mosaic: 0.9278886979329984
  mixup: 0.08598744863101347
  degrees: 7.142500904695323
  scale: 0.3370923531181857
  shear: 1.4446355785997234
  hsv_h: 0.12436059484999436
  hsv_s: 0.21036293426832842
  hsv_v: 0.24750678002381593
  translate: 0.11690807067290405
  lr0: 0.006660428569568175
  lrf: 0.4504075638333678
  momentum: 0.8813827330930218
  weight_decay: 0.00016069339868104082
  warmup_epochs: 5
  warmup_momentum: 0.9280056873353899
  warmup_bias_lr: 0.48913347988315226
  box: 0.5184605686512475
  cls: 0.5522794440735213
  dfl: 1.0038762284088887
  dropout: 0.30659781843342954
  patience: 9


In [5]:
import pandas as pd
df = study.trials_dataframe()
df.to_csv("optuna_yolov8_trials.csv", index=False)
df.head()

,number,value,datetime_start,datetime_complete,duration,params_batch,params_box,params_cls,params_degrees,params_dfl,...,params_mosaic,params_patience,params_scale,params_shear,params_translate,params_warmup_bias_lr,params_warmup_epochs,params_warmup_momentum,params_weight_decay,state
0,0,0.532076,2025-04-22 21:33:12.258150,2025-04-22 21:42:40.746194,0 days 00:09:28.488044,16,0.371879,0.753107,16.456212,1.846768,...,0.892905,16,0.507209,1.081236,0.027474,0.462205,1,0.558138,0.000250,COMPLETE
1,1,NaN,2025-04-22 21:42:42.214411,2025-04-22 21:44:31.890369,0 days 00:01:49.675958,32,0.658005,0.706853,16.729858,1.351147,...,0.909352,20,0.526218,0.433808,0.014710,0.224872,0,0.689629,0.000978,FAIL
2,2,NaN,2025-04-22 21:44:33.798840,2025-04-22 21:44:34.365449,0 days 00:00:00.566609,32,0.173239,0.733667,13.512678,2.425478,...,0.852539,15,0.421851,4.408831,0.127592,0.300933,2,0.801175,0.000595,FAIL
3,3,NaN,2025-04-22 21:44:34.659104,2025-04-22 21:44:35.091053,0 days 00:00:00.431949,16,0.311877,0.658602,2.489363,2.169584,...,0.543400,7,0.450226,1.529266,0.111286,0.201073,2,0.760340,0.000252,FAIL
4,4,NaN,2025-04-22 21:44:35.326242,2025-04-22 21:44:35.656614,0 days 00:00:00.330372,32,0.694209,0.496553,16.868391,1.812695,...,0.981591,10,0.543209,1.638810,0.015929,0.362060,0,0.906262,0.000937,FAIL


In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

train_args = {
    "data": './fire and smoke.v3i.yolov8/data.yaml',
    "epochs": 150,
    "time": None,
    "patience": 10,
    "batch": 8,
    "imgsz": 640,
    "save": True,
    "save_period": -1,
    "cache": False,
    "device": None,
    "workers": 8,
    "project": None,
    "name": "train",
    "exist_ok": False,
    "pretrained": True,
    "optimizer": "auto",
    "verbose": True,
    "seed": 0,
    "deterministic": True,
    "single_cls": False,
    "rect": False,
    "cos_lr": False,
    "close_mosaic": 10,
    "resume": False,
    "amp": True,
    "fraction": 1.0,
    "profile": False,
    "freeze": None,
    "multi_scale": False,
    "overlap_mask": True,
    "mask_ratio": 4,
    "dropout": 0.0,
    "val": True,
    "split": "val",
    "save_json": False,
    "save_hybrid": False,
    "conf": None,
    "iou": 0.7,
    "max_det": 300,
    "half": False,
    "dnn": False,
    "plots": True,
    "source": None,
    "vid_stride": 1,
    "stream_buffer": False,
    "visualize": False,
    "augment": False,
    "agnostic_nms": False,
    "classes": None,
    "retina_masks": False,
    "embed": None,
    "show": False,
    "save_frames": False,
    "save_txt": False,
    "save_conf": False,
    "save_crop": False,
    "show_labels": True,
    "show_conf": True,
    "show_boxes": True,
    "line_width": None,
    "format": "torchscript",
    "keras": False,
    "optimize": False,
    "int8": False,
    "dynamic": False,
    "simplify": False,
    "opset": None,
    "workspace": 4,
    "nms": False,
    "lr0": 0.01,
    "lrf": 0.01,
    "momentum": 0.937,
    "weight_decay": 0.0005,
    "warmup_epochs": 3.0,
    "warmup_momentum": 0.8,
    "warmup_bias_lr": 0.1,
    "box": 7.5,
    "cls": 0.5,
    "dfl": 1.5,
    "pose": 12.0,
    "kobj": 1.0,
    "label_smoothing": 0.0,
    "nbs": 64,
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 0.0,
    "translate": 0.1,
    "scale": 0.5,
    "shear": 0.0,
    "perspective": 0.0,
    "flipud": 0.0,
    "fliplr": 0.5,
    "bgr": 0.0,
    "mosaic": 1.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
    "auto_augment": "randaugment",
    "erasing": 0.4,
    "crop_fraction": 1.0,
    "cfg": None
}

model.train(**train_args)


New https://pypi.org/project/ultralytics/8.3.115 available  Update with 'pip install -U ultralytics'
WARNING  'save_hybrid' is deprecated and will be removed in in the future.
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.107  Python-3.12.7 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./fire and smoke.v3i.yolov8/data.yaml, epochs=150, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train51, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sourc

train: Scanning C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\train\labels.cache... 2712 images, 5 backgrounds, 1 corrupt: 100%|██████████| 2713/2713 [00:00<?, ?it/s]

train: WARNING  C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\train\images\fire-and-truck-desert-an-aerial-view-of-a-very-small-fire-on-the-side-of-a-road-in-a-rural-desert-landscape-a-green-tractor-stands-by-as-the-fire-burns_bdglz60p8__M0000_jpg.rf.b186e9bec055032ed74041cc43e162d1.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'C:\\Users\\G15\\Desktop\\FireDetectionSystem-main\\fire and smoke.v3i.yolov8\\train\\images\\fire-and-truck-desert-an-aerial-view-of-a-very-small-fire-on-the-side-of-a-road-in-a-rural-desert-landscape-a-green-tractor-stands-by-as-the-fire-burns_bdglz60p8__M0000_jpg.rf.b186e9bec055032ed74041cc43e162d1.jpg'



val: Scanning C:\Users\G15\Desktop\FireDetectionSystem-main\fire and smoke.v3i.yolov8\valid\labels.cache... 728 images, 1 backgrounds, 0 corrupt: 100%|██████████| 728/728 [00:00<?, ?it/s]


Plotting labels to runs\detect\train51\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train51
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     52/150      1.22G      1.591       1.53      1.585         11        640: 100%|██████████| 339/339 [00:35<00:00,  9.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:05<00:00,  9.08it/s]


                   all        728       1524       0.61      0.491      0.548      0.255
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 42, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 0.597 hours.
Optimizer stripped from runs\detect\train51\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train51\weights\best.pt, 6.2MB

Validating runs\detect\train51\weights\best.pt...
Ultralytics 8.3.107  Python-3.12.7 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:05<00:00,  8.62it/s]


                   all        728       1524      0.598      0.517      0.561      0.267
                  fire        581        908      0.611      0.643      0.668      0.344
                 smoke        396        616      0.585      0.391      0.453      0.191
Speed: 0.3ms preprocess, 1.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\train51


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F257AEBB90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0